In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
%reload_ext autoreload
%autoreload 2

In [13]:
# load model 

import torch
import dnnlib
from training.networks_stylegan3 import Generator

from legacy import load_network_pkl
device = torch.device('cuda', 0)
gpath = "pretrained/stylegan3-r-ffhq-1024x1024.pkl" # pretrained/stylegan3-t-ffhq-1024x1024.pkl
with open(gpath, 'rb') as f:
    G_old = load_network_pkl(f)['G_ema'].to(device)


In [8]:
#G_kwargs = dnnlib.EasyDict(class_name="training.networks_stylegan3.Generator", z_dim=512, w_dim=512, mapping_kwargs=dnnlib.EasyDict())
device = torch.device('cuda', 0)
training_set = dnnlib.EasyDict(
    label_dim = 0,
    num_channels = 3,
)
G_kwargs = dnnlib.EasyDict(G_old.init_kwargs)
G_kwargs.class_name = "training.networks_stylegan3.Generator"
G_kwargs.img_resolution = 1024
# img_resolution = 1024
# common_kwargs_4k = dict(c_dim=training_set.label_dim, img_resolution=4096, img_channels=training_set.num_channels)
# common_kwargs = dict(c_dim=training_set.label_dim, img_resolution=img_resolution, img_channels=training_set.num_channels)
G: Generator = dnnlib.util.construct_class_by_name(**G_kwargs, actual_resolution=1024).train().requires_grad_(False).to(device) # subclass of torch.nn.Module

cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])


In [9]:
before = G.synthesis.get_dict_of_up_filters()

In [10]:
from torch_utils import misc
misc.copy_params_and_buffers(G_old, G, require_all=False, allow_ignore_different_shapes=True, verbose=True)

Copying synthesis.input.weight: got torch.Size([1024, 1024]), expected torch.Size([1024, 1024])
Copying synthesis.input.affine.weight: got torch.Size([4, 512]), expected torch.Size([4, 512])
Copying synthesis.input.affine.bias: got torch.Size([4]), expected torch.Size([4])
Copying synthesis.L0_36_1024.weight: got torch.Size([1024, 1024, 1, 1]), expected torch.Size([1024, 1024, 1, 1])
Copying synthesis.L0_36_1024.bias: got torch.Size([1024]), expected torch.Size([1024])
Copying synthesis.L0_36_1024.affine.weight: got torch.Size([1024, 512]), expected torch.Size([1024, 512])
Copying synthesis.L0_36_1024.affine.bias: got torch.Size([1024]), expected torch.Size([1024])
Copying synthesis.L1_36_1024.weight: got torch.Size([1024, 1024, 1, 1]), expected torch.Size([1024, 1024, 1, 1])
Copying synthesis.L1_36_1024.bias: got torch.Size([1024]), expected torch.Size([1024])
Copying synthesis.L1_36_1024.affine.weight: got torch.Size([1024, 512]), expected torch.Size([1024, 512])
Copying synthesis.L1

In [80]:
after = G.synthesis.get_dict_of_up_filters()

In [84]:
G.synthesis.reconfigure_network(256)

cutoffs=array([  2.        ,   2.82842712,   4.        ,   5.65685425,
         8.        ,  11.3137085 ,  16.        ,  22.627417  ,
        32.        ,  45.254834  ,  64.        ,  90.50966799,
       128.        , 128.        , 128.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])


In [85]:
after_reconfigure = G.synthesis.get_dict_of_up_filters()

In [44]:
!source sh/setup40.sh
# random input test
import torch
import numpy as np
z = torch.from_numpy(np.random.randn(1, 512).astype(np.float32)).to(device)
c = torch.randn(1, G.c_dim, device=device)

w = G.mapping(z, c)
print(w.shape)
img = G.synthesis(w)
img.shape

Autoloading gmp/6.2.1/gcc-7.5.0-2px2tki
Autoloading mpc/1.1.0/gcc-7.5.0-osodvlz
Autoloading mpfr/4.1.0/gcc-7.5.0-ublhnmp
Autoloading zlib/1.2.11/gcc-7.5.0-h3xiplk
Autoloading zstd/1.5.0/gcc-7.5.0-qsdxtru
Autoloading cuda/10.2.89/gcc-7.5.0-jr6kobf
Autoloading libxml2/2.9.9/gcc-7.5.0-ouvtc2l
Autoloading libiconv/1.16/gcc-7.5.0-pndwbk6
Autoloading xz/5.2.4/gcc-7.5.0-55h4brm
Autoloading zlib/1.2.11/gcc-7.5.0-dsnnbcq
Autoloading libxml2/2.9.12/gcc-7.5.0-i2j2pf5
Autoloading libiconv/1.16/gcc-7.5.0-cvte7wx
Autoloading xz/5.2.5/gcc-7.5.0-sytpwnz
Autoloading zlib/1.2.11/gcc-7.5.0-h3xiplk
>torch.Size([1, 16, 512])
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 102

torch.Size([1, 3, 256, 256])

In [6]:
from util import patch_util
patches = patch_util.generate_full_from_patches(1024, 256)
patches[5]

((256, 512, 256, 512),
 tensor([[ 0.2500,  0.0000, -0.1250],
         [ 0.0000,  0.2500, -0.1250],
         [ 0.0000,  0.0000,  1.0000]]))

In [40]:
len(patches)

16

In [19]:
import torchinfo
summary = torchinfo.summary(G, input_data=[z, c], col_names = ["input_size", "output_size", "num_params", "mult_adds"], depth=8)
# write to text file
location = "test/arhitecture.txt"
with open(location, 'w') as f:
    f.write(str(summary))

freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 1024, 36, 36])


In [29]:
def get_diff_up_filter(original, new, verbose=False):
    non_equal = []
    for key, value in original.items():
        if value is None and new[key] is None:
            if verbose:
                print(key, True)
            continue
        if value.shape != new[key].shape:
            if verbose:
                print(key, False)
            non_equal.append(key)
            continue
        eq = torch.all(torch.eq(new[key], value))
        if not eq:
            non_equal.append(key)
        if verbose:
            print(key, eq)
    return non_equal

In [39]:
device = torch.device('cuda', 0)
training_set = dnnlib.EasyDict(
    label_dim = 0,
    num_channels = 3,
)
G_kwargs = dnnlib.EasyDict(G_old.init_kwargs)
G_kwargs.class_name = "training.networks_stylegan3.Generator"
G_kwargs.img_resolution = 256 # 256, 1024 or 4096
G: Generator = dnnlib.util.construct_class_by_name(**G_kwargs, actual_resolution=1024).train().requires_grad_(False).to(device) # subclass of torch.nn.Module

def to_cpu(x):
    return x.cpu() if x is not None else None

def get_tensor_values(model):
    # get upfilter from model.synthesis.layer_names
    return {name:to_cpu(getattr(model.synthesis, name).up_filter) for name in model.synthesis.layer_names}

def get_dict_to_str(d):
    final = ""
    for key, value in d.items():
        final += f"{key}:\n {value}\n"
    return final

original_values = get_tensor_values(G_old)
with open(f"test/arhitecture_{G_kwargs.img_resolution}_original.txt", 'w') as f:
    f.write(get_dict_to_str(get_tensor_values(G_old)))

with open(f"test/arhitecture_{G_kwargs.img_resolution}_before.txt", 'w') as f:
    f.write(get_dict_to_str(get_tensor_values(G)))

from torch_utils import misc
misc.copy_params_and_buffers(G_old, G, require_all=False, allow_ignore_different_shapes=True, verbose=False)
with open(f"test/arhitecture_{G_kwargs.img_resolution}.txt", 'w') as f:
    f.write(str(G))
with open(f"test/arhitecture_{G_kwargs.img_resolution}_after.txt", 'w') as f:
    f.write(get_dict_to_str(G.synthesis.get_dict_of_up_filters()))

after_values = get_tensor_values(G)

non_eq = get_diff_up_filter(original_values, after_values, verbose=False)
print(f"non equal: {non_eq}")

cutoffs=array([  2.        ,   2.82842712,   4.        ,   5.65685425,
         8.        ,  11.3137085 ,  16.        ,  22.627417  ,
        32.        ,  45.254834  ,  64.        ,  90.50966799,
       128.        , 128.        , 128.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=1024
Reminder: using all at 256 except in_channels=1024 and out_channels=

In [40]:
G.synthesis.reconfigure_network(256)

with open(f"test/arhitecture_{G_kwargs.img_resolution}_after_rc.txt", 'w') as f:
    f.write(get_dict_to_str(G.synthesis.get_dict_of_up_filters()))

non_eq2 = get_diff_up_filter(after_values, get_tensor_values(G), verbose=True)
print(f"non equal after reconfigure: {non_eq}")

non_eq3 = get_diff_up_filter(original_values, get_tensor_values(G), verbose=True)
print(f"non equal after reconfigure: {non_eq}, original values")

cutoffs=array([  2.        ,   2.82842712,   4.        ,   5.65685425,
         8.        ,  11.3137085 ,  16.        ,  22.627417  ,
        32.        ,  45.254834  ,  64.        ,  90.50966799,
       128.        , 128.        , 128.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])
L0_36_1024 tensor(True)
L1_36_1024 tensor(True)
L2_52_1024 tensor(True)
L3_52_1024 tensor(True)
L4_84_1024 tensor(True)
L5_148_1024 tensor(False)
L6_148_1024 tensor(False)
L7_276_645 tensor(True)
L8_276_406 tensor(False)
L9_532_256 tensor(True)
L10_1044_161 tensor(False)
L11_1044_102 tensor(False)
L12_1044_64 tensor(False)
L13_1024_64 tensor(True)
L14_1024_3 True
non equal after reconfigure: ['L2_52_1024', 'L3_52_1024', 'L4_84_1024', 'L9_532_256']
L0_36_1024 tensor(True)
L1_36_1024 tensor(True)
L2_52_

In [46]:
### FOR FULL RES PATCHES

import torch
import numpy as np
seeds = list(range(16))
from gen_images import make_transform
device = torch.device('cuda', 0)
#crop_params = (0, 256, 0, 256)
# transform_left_top = torch.tensor([ [ 0.2500,  0.0000, -0.3750],
#                                     [ 0.0000,  0.2500, -0.3750],
#                                     [ 0.0000,  0.0000,  1.0000]]).unsqueeze(0).to(device)
transform_left_top = patches[5][1].unsqueeze(0).to(device) # should be eye-ish
print(f"Used crop params:{patches[5][0]}")
# transform_left_top = None # Use when we want scale 1

from util import renormalize
def do_full(G, seeds, truncation_psi=0.5, outdir = 'test', extra_folder= None):

    translate = (0, 0)
    rotate = 0

    noise_mode = 'const'
    label = None
    import PIL
    import os

    if extra_folder != None:
        final_dir = f'{outdir}/{extra_folder}/'
    else:
        final_dir = f'{outdir}/'

    if not os.path.exists(final_dir):
        os.makedirs(final_dir)

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
        ws = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)
        # Construct an inverse rotation/translation matrix and pass to the generator.  The
        # generator expects this matrix as an inverse to avoid potentially failing numerical
        # operations in the network.
        full = torch.zeros([1, 3, G.actual_resolution, G.actual_resolution])
        patches = patch_util.generate_full_from_patches(G.actual_resolution, G.img_resolution)
        for bbox, transform in patches:
            img = patch_util.scale_condition_wrapper(G, ws, transform[None].cuda(), noise_mode='const', force_fp32=True)
            full[:, :, bbox[0]:bbox[1], bbox[2]:bbox[3]] = img
        renormalize.as_image(full[0]).save(f'{final_dir}{truncation_psi}seed{seed:04d}.png')
        # PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

do_full(G, seeds, 1, extra_folder="256_rc1_f")

Used crop params:(256, 512, 256, 512)
Generating image for seed 0 (0/16) ...
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 1024, 36, 36])
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 1024, 36, 36])
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms

In [48]:
import torch
import numpy as np
seeds = list(range(16))
from gen_images import make_transform
device = torch.device('cuda', 0)
#crop_params = (0, 256, 0, 256)
# transform_left_top = torch.tensor([ [ 0.2500,  0.0000, -0.3750],
#                                     [ 0.0000,  0.2500, -0.3750],
#                                     [ 0.0000,  0.0000,  1.0000]]).unsqueeze(0).to(device)
transform_left_top = patches[5][1].unsqueeze(0).to(device) # should be eye-ish
print(f"Used crop params:{patches[5][0]}")
# transform_left_top = None # Use when we want scale 1
def do(G, seeds, truncation_psi=0.5, outdir = 'test', transform = None, extra_folder= None):

    translate = (0, 0)
    rotate = 0

    noise_mode = 'const'
    label = None
    import PIL
    import os

    if extra_folder != None:
        final_dir = f'{outdir}/{extra_folder}/'
    else:
        final_dir = f'{outdir}/'

    if not os.path.exists(final_dir):
        os.makedirs(final_dir)

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)

        # Construct an inverse rotation/translation matrix and pass to the generator.  The
        # generator expects this matrix as an inverse to avoid potentially failing numerical
        # operations in the network.
        if hasattr(G.synthesis, 'input'):
            m = make_transform(translate, rotate)
            m = np.linalg.inv(m)
            G.synthesis.input.transform.copy_(torch.from_numpy(m))

        torch.set_grad_enabled(False)
        img = G(z, label, truncation_psi=truncation_psi, transform=transform, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        print(img.shape)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{final_dir}{truncation_psi}seed{seed:04d}.png')

do(G, seeds, 1, transform=None, extra_folder="256_rc1_s1")

Used crop params:(256, 512, 256, 512)
Generating image for seed 0 (0/16) ...
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 1024, 36, 36])
torch.Size([1, 256, 256, 3])
Generating image for seed 1 (1/16) ...
freqs.shape=torch.Size([1, 1024, 2]) | phases.shape=torch.Size([1, 1024])
grids.shape=torch.Size([1, 36, 36, 2]) | transforms.shape=torch.Size([1, 3, 3])
x.shape=torch.Size([1, 36, 36, 1024]) * amplitudes.shape=torch.Size([1, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) @ weight.t().shape=torch.Size([1024, 1024])
x.shape=torch.Size([1, 36, 36, 1024]) after @
x.shape=torch.Size([1, 1024, 36, 36])
torch.Size([1, 256, 256, 3])
Generating image for seed 2 

### Directly compute?

In [51]:
import dnnlib
import torch
#G_kwargs = dnnlib.EasyDict(class_name="training.networks_stylegan3.Generator", z_dim=512, w_dim=512, mapping_kwargs=dnnlib.EasyDict())
device = torch.device('cuda', 0)
training_set = dnnlib.EasyDict(
    label_dim = 0,
    num_channels = 3,
)
G_kwargs = dnnlib.EasyDict(G_old.init_kwargs)
G_kwargs.class_name = "training.networks_stylegan3.Generator"
G_kwargs.img_resolution = 1024
# img_resolution = 1024
# common_kwargs_4k = dict(c_dim=training_set.label_dim, img_resolution=4096, img_channels=training_set.num_channels)
# common_kwargs = dict(c_dim=training_set.label_dim, img_resolution=img_resolution, img_channels=training_set.num_channels)
G: Generator = dnnlib.util.construct_class_by_name(**G_kwargs).train().requires_grad_(False).to(device) # subclass of torch.nn.Module

cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])


65536

In [57]:
# def compute_stuff_for_resolution(self, img_resolution, channel_base, channel_max):
# return channels, sizes, sampling_rates, cutoffs, half_widths
channels, sizes, sampling_rates, cutoffs, half_widths = G.synthesis.compute_stuff_for_resolution(1024, G.init_kwargs.channel_base, G.init_kwargs.channel_max )

cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])


In [59]:
resolutions = [256, 1024, 4096]
for res in resolutions:
    channels, sizes, sampling_rates, cutoffs, half_widths = G.synthesis.compute_stuff_for_resolution(res, G.init_kwargs.channel_base, G.init_kwargs.channel_max )
    with open(f"test/resolution_config{res}.txt", "w") as f:
        f.write(f"channels: {channels}\n")
        f.write(f"sizes: {sizes}\n")
        f.write(f"sampling_rates: {sampling_rates}\n")
        f.write(f"cutoffs: {cutoffs}\n")
        f.write(f"half_widths: {half_widths}\n")

cutoffs=array([  2.        ,   2.82842712,   4.        ,   5.65685425,
         8.        ,  11.3137085 ,  16.        ,  22.627417  ,
        32.        ,  45.254834  ,  64.        ,  90.50966799,
       128.        , 128.        , 128.        ])
cutoffs=array([  2.        ,   3.1748021 ,   5.0396842 ,   8.        ,
        12.69920842,  20.1587368 ,  32.        ,  50.79683366,
        80.63494719, 128.        , 203.18733465, 322.53978877,
       512.        , 512.        , 512.        ])
cutoffs=array([2.00000000e+00, 3.56359487e+00, 6.34960421e+00, 1.13137085e+01,
       2.01587368e+01, 3.59187855e+01, 6.40000000e+01, 1.14035036e+02,
       2.03187335e+02, 3.62038672e+02, 6.45079578e+02, 1.14940114e+03,
       2.04800000e+03, 2.04800000e+03, 2.04800000e+03])


In [58]:
channels

array([1024., 1024., 1024., 1024., 1024., 1024., 1024.,  645.,  406.,
        256.,  161.,  102.,   64.,   64.,    3.])